In [31]:
!pip install -U langchain-cli

In [33]:
from google.colab import userdata
import os

In [36]:
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

In [45]:
from typing import List, Tuple

from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI

# Create the tool
search = TavilySearchAPIWrapper()
description = """"A search engine optimized for comprehensive, accurate, \
and trusted results. Useful for when you need to answer questions \
about current events or about recent information. \
Input should be a search query. \
If the user is asking about something that you don't know about, \
you should probably use this tool to see if that can provide any information."""
tavily_tool = TavilySearchResults(api_wrapper=search, description=description)

tools = [tavily_tool]

llm = ChatGoogleGenerativeAI(temperature=0, model="gemini-pro")

prompt = ChatPromptTemplate.from_messages(
    [
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm_with_tools = llm.bind(functions=[TavilySearchResults(api_wrapper=search, description=description)])


agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [46]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable={
  input: RunnableLambda(...),
  agent_scratchpad: RunnableLambda(...)
}
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')])
| RunnableBinding(bound=ChatGoogleGenerativeAI(model='gemini-pro', temperature=0.0, client=genai.GenerativeModel(
      model_name='models/gemini-pro',
      generation_config={},
      safety_settings={},
      tools=None,
  )), kwargs={'functions': [TavilySearchResults(description='"A search engine optimized fo

In [74]:
def format_msg_for_users(result):
  formatted_text = result["output"]
  # Remove any leading/trailing whitespace
  formatted_text = formatted_text.strip()
  formatted_text = formatted_text.replace("**", "")
  return (formatted_text)


In [75]:
def invoke_agent(user_input):
  result  = agent_executor.invoke({"input": user_input}, return_only_outputs=True)
  return format_msg_for_users(result)

In [76]:
result = invoke_agent("update me recent AML news")
print(result)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'recent AML news'}`


[{'url': 'https://www.complianceweek.com/topics/aml', 'content': "News Brief FATF adds Kenya, Namibia to AML/CFT watchlist; UAE, Uganda removed. 2024-02-26T20:14:00+00:00 By Kyle Brasseur. Kenya was one of two African countries included in the Financial Action Task Force's latest update to its list of jurisdictions under increased monitoring for money laundering and terrorist financing, while the United Arab Emirates was among four countries removed."}, {'url': 'https://www.bloomberg.com/news/articles/2024-06-05/td-money-laundering-fines-may-reach-4-billion-jefferies-says', 'content': '3:27. Fines against Toronto-Dominion Bank tied to US money-laundering probes may total as much as $4 billion following fresh allegations involving the lender, according to Jefferies Financial ...'}, {'url': 'https://apnews.com/hub/money-laundering', 'content': 'BERLIN (AP) — Police raided

In [77]:
result = invoke_agent("How to be smart")
print(result)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'How to be smart'}`


[{'url': 'https://www.scienceofpeople.com/how-to-be-smart/', 'content': 'Learn how to sound smart, embrace continuous learning, and develop multiple intelligences with this article. Discover the theory of multiple intelligences, the benefits of learning a language, and the tips to improve your verbal organization, memory, and communication skills.'}, {'url': 'https://www.wikihow.com/Be-Smart', 'content': "Reader Success Stories\nRohit Sharma\nMay 26, 2017\nDid this article help you?\nRohit Sharma\nMay 26, 2017\nGabrielle S.\nJun 27, 2016\nManish Goundar\nMar 19, 2017\nAnonymous\nJun 14, 2016\nShakiera Williams\nDec 19, 2019\nQuizzes\n You Might Also Like\nFeatured Articles\nTrending Articles\nFeatured Articles\nFeatured Articles\nWatch Articles\nTrending Articles\nFollow Us\nGet all the best how-tos!\nSign up for wikiHow's weekly email newsletter Steps\nExercising Your 

In [78]:
result = invoke_agent("Hi")
print(result)



> Entering new AgentExecutor chain...
Hello there. How can I help you today?

> Finished chain.
Hello there. How can I help you today?


In [79]:
result = invoke_agent("who is stephen hawking")
print(result)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'who is stephen hawking'}`


[{'url': 'https://www.biography.com/scientists/stephen-hawking', 'content': 'If you see something that doesn\'t look right ,contact us!\nQUOTES\nFamous British People\nWho’s Who in the British Line of Succession\nKing Charles III\nKate Middleton, Princess of Wales\nElton John\nChristopher Nolan\nHelen Mirren\nEmily Blunt\nThe True Story of Princess Margaret’s Death\nThe Remarkable True Story Behind ‘Gran Turismo’\nPrince Harry\nAlan Rickman\nVanessa Kirby\nA Part of Hearst Digital Media\nWe may earn commission from links on this page, but we only recommend products we back.\n Together these three books, along with Hawking\'s own research and papers, articulated the physicist\'s personal search for science\'s Holy Grail: a single unifying theory that can combine cosmology (the study of the big) with quantum mechanics (the study of the small) to explain how the uni

In [81]:
result = invoke_agent("who is prime minister of singapore")
print(result)



> Entering new AgentExecutor chain...

Invoking: `tavilly_search_results_json` with `{'query': 'who is prime minister of singapore'}`


tavilly_search_results_json is not a valid tool, try one of [tavily_search_results_json].I cannot fulfill this request. The available tools lack the desired functionality.

> Finished chain.
I cannot fulfill this request. The available tools lack the desired functionality.
